In [1]:
import numpy as np
import pandas as pd
import pickle
from tempfile import mkdtemp
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import r2_score,make_scorer
from sklearn.preprocessing import StandardScaler,scale as scaler
from sklearn.model_selection import GridSearchCV
import math
from IPython.display import display, HTML

from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.recurrent import LSTM, GRU
from keras.layers import Convolution1D, MaxPooling1D, LeakyReLU
from keras.optimizers import RMSprop, Adam, SGD, Nadam
from keras.wrappers.scikit_learn import KerasRegressor
from keras import regularizers
from keras.layers.normalization import BatchNormalization
from collections import namedtuple
from keras import losses
from sklearn.pipeline import Pipeline

Using TensorFlow backend.


In [2]:
def split_into_chunks(data, train, predict, step, binary=True, scale=False):
    X, Y = [], []
    for i in range(0, len(data), step):
        try:
            x_i = data[i:i+train]
            y_i = data[i+train+predict]
            
            # Use it only for daily return time series
            if binary:
                if y_i > 0.:
                    y_i = [1., 0.]
                else:
                    y_i = [0., 1.]

                if scale: x_i = scaler(x_i)
                
            else:
                
                timeseries =np.array(data[i:i+train+predict])
                
                if scale: 
                    timeseries = scaler(timeseries)
                x_i = timeseries[:-1]
                y_i = timeseries[-1]
            
        except  Exception as e:
            print (str(e))
            break

        X.append(x_i)
        Y.append(y_i)

    return X, Y

def train_test_split(X, y, train_percentage):
    rows=len(X)
    train_size=int(rows*train_percentage)
    X_train=X[:train_size]
    y_train=y[:train_size]
    X_test=X[train_size:]
    y_test=y[train_size:]
    return X_train,y_train,X_test,y_test


In [3]:
Dataset=namedtuple('Dataset','exchange df')
DatasetMLModel= namedtuple('DatasetMLModel','exchange  train_size tscv_split test_size X_train y_train X_test y_test scaler_features scaler_target')
Regressor= namedtuple('Regressor','name regressor_class params type')
FeatureSelection= namedtuple('FeatureSelection','dataset regressor params RFECV')
TRAIN_PERCENTAGE=0.8
TRAIN_SIZE_FEATURES = 20
EVALUATION_PERCENT=0.2
TARGET_TIME = 1
LAG_SIZE = 1
EMB_SIZE = 1


with open('datasets/log_divided_close_datasets.pkl', 'rb') as input4:
    log_divided_close_datasets = pickle.load(input4)


    
dataset=log_divided_close_datasets['btc_brl']
data=dataset.df['log_return']

rows,=data.shape
train_size=int(rows*TRAIN_PERCENTAGE)
scaler = StandardScaler()
scaler.fit(data[:train_size+TRAIN_SIZE_FEATURES].values.reshape(-1, 1))
data=scaler.transform(data.values.reshape(-1, 1))
data=data.reshape(1,-1).tolist()[0]

In [4]:
data[20]

-4.281966288117148

In [5]:

X,Y =split_into_chunks(data, TRAIN_SIZE_FEATURES,TARGET_TIME, LAG_SIZE, binary=False, scale=False)
X, Y = np.array(X), np.array(Y)
X_train,y_train,X_test,y_test=train_test_split(X, Y, TRAIN_PERCENTAGE)

list index out of range


In [6]:
y_train


array([-4.28196629,  6.70878824, -5.11171855, ...,  0.0928846 ,
       -0.09352972,  0.06764178])

In [7]:
tscv_split=int(rows//(train_size*EVALUATION_PERCENT)-1)

In [8]:
def create_model_deep_learning():  
    model = Sequential()
    model.add(Dense(64, input_shape=(TRAIN_SIZE_FEATURES,),activity_regularizer=regularizers.l2(0.01))) 
    model.add(BatchNormalization()) 
    model.add(LeakyReLU()) 
    model.add(Dense(16,activity_regularizer=regularizers.l2(0.01))) 
    model.add(BatchNormalization()) 
    model.add(LeakyReLU()) 
    model.add(Dense(1)) 
    model.add(Activation('linear'))
    opt = Nadam(lr=0.001)
    model.compile(optimizer='adam', 
              loss='mse')
    return model

def create_model_deep_learning2(size1=64,size2=64,size3=64,activation='relu', lr=0.01,loss=losses.mse):  
    model = Sequential()
    model.add(Dense(size1, input_shape=(TRAIN_SIZE_FEATURES,),activation=activation)) 
    model.add(Dense(size2,activation=activation) )
    model.add(Dense(size3,activation=activation) )
    model.add(Dense(1)) 
    model.add(Activation('linear'))
    opt = Adam(lr=lr)
    model.compile(optimizer=opt, 
              loss=loss)
    return model

In [9]:
def performance_metric(y_true, y_predict):
    """ Calculates and returns the performance score between 
        true and predicted values based on the metric chosen. """
    score = r2_score(y_true,y_predict)
    return score

In [10]:
model = KerasRegressor(build_fn=create_model_deep_learning2)

In [11]:
batch_size = [10,50]
epochs = [5,10,20,30,40,50]
lr=[0.01,0.001,0.1]
size1=[64,128]
size2=[64,128]
size3=[64,128]
activation=['relu', 'sigmoid','tanh','linear']
loss=[losses.logcosh,losses.mse,losses.mae,losses.mape]

param_grid = dict(keras_model=[KerasRegressor(build_fn=create_model_deep_learning2)],
                  keras_model__batch_size=batch_size,
                  keras_model__epochs=epochs,
#                  keras_model__lr=lr,
                  keras_model__size1=size1,
                  keras_model__size2=size2,
                  keras_model__size3=size3,
                  keras_model__activation=activation,
                  keras_model__loss=loss
                 )

param_grid

{'keras_model': [<keras.wrappers.scikit_learn.KerasRegressor at 0x117b872b0>],
 'keras_model__activation': ['relu', 'sigmoid', 'tanh', 'linear'],
 'keras_model__batch_size': [10, 50],
 'keras_model__epochs': [5, 10, 20, 30, 40, 50],
 'keras_model__loss': [<function keras.losses.logcosh>,
  <function keras.losses.mean_squared_error>,
  <function keras.losses.mean_absolute_error>,
  <function keras.losses.mean_absolute_percentage_error>],
 'keras_model__size1': [64, 128],
 'keras_model__size2': [64, 128],
 'keras_model__size3': [64, 128]}

In [ ]:
cachedir = mkdtemp()
estimators= [('keras_model',KerasRegressor(build_fn=create_model_deep_learning2))]
params=[param_grid,]
pipe = Pipeline(estimators, memory=cachedir)
pipe.get_params().keys()

dict_keys(['memory', 'steps', 'keras_model', 'keras_model__build_fn'])

In [ ]:
scoring_fnc = make_scorer(performance_metric)
cvts=TimeSeriesSplit(n_splits=tscv_split)
grid = GridSearchCV(estimator=pipe, 
                    param_grid=params,
                    scoring=scoring_fnc,
                    cv=cvts,
                    error_score=np.NaN,
                    n_jobs=1
                   )  
grid = grid.fit(X_train,y_train )

Epoch 1/5
222/222 [==============================] - 0s 1ms/step - loss: 0.8223
Epoch 2/5
222/222 [==============================] - 0s 174us/step - loss: 0.6838
Epoch 3/5
222/222 [==============================] - 0s 174us/step - loss: 0.6103
Epoch 4/5
222/222 [==============================] - 0s 171us/step - loss: 0.6020
Epoch 5/5
222/222 [==============================] - 0s 183us/step - loss: 0.5549
Epoch 1/5
443/443 [==============================] - 0s 940us/step - loss: 0.5122
Epoch 2/5
443/443 [==============================] - 0s 174us/step - loss: 0.4650
Epoch 3/5
443/443 [==============================] - 0s 167us/step - loss: 0.4303
Epoch 4/5
443/443 [==============================] - 0s 167us/step - loss: 0.3969
Epoch 5/5
443/443 [==============================] - 0s 167us/step - loss: 0.4125
Epoch 1/5
664/664 [==============================] - 0s 595us/step - loss: 0.4377
Epoch 2/5
664/664 [==============================] - 0s 160us/step - loss: 0.3973
Epoch 3/5
664/664 

1106/1106 [==============================] - 0s 189us/step - loss: 0.3085
Epoch 1/5
222/222 [==============================] - 1s 4ms/step - loss: 0.7438
Epoch 2/5
222/222 [==============================] - 0s 188us/step - loss: 0.6879
Epoch 3/5
222/222 [==============================] - 0s 189us/step - loss: 0.6289
Epoch 4/5
222/222 [==============================] - 0s 196us/step - loss: 0.5271
Epoch 5/5
222/222 [==============================] - 0s 196us/step - loss: 0.4911
Epoch 1/5
443/443 [==============================] - 1s 2ms/step - loss: 0.5466
Epoch 2/5
443/443 [==============================] - 0s 195us/step - loss: 0.4789
Epoch 3/5
443/443 [==============================] - 0s 196us/step - loss: 0.4577
Epoch 4/5
443/443 [==============================] - 0s 197us/step - loss: 0.4322
Epoch 5/5
443/443 [==============================] - 0s 191us/step - loss: 0.4172
Epoch 1/5
664/664 [==============================] - 1s 2ms/step - loss: 0.4356
Epoch 2/5
664/664 [===========

1106/1106 [==============================] - 0s 228us/step - loss: 0.3015
Epoch 5/5
1106/1106 [==============================] - 0s 222us/step - loss: 0.3025
Epoch 1/5
222/222 [==============================] - 1s 7ms/step - loss: 3.7100
Epoch 2/5
222/222 [==============================] - 0s 216us/step - loss: 2.8195
Epoch 3/5
222/222 [==============================] - 0s 222us/step - loss: 2.4134
Epoch 4/5
222/222 [==============================] - 0s 216us/step - loss: 2.9171
Epoch 5/5
222/222 [==============================] - 0s 219us/step - loss: 2.2546
Epoch 1/5
443/443 [==============================] - 2s 3ms/step - loss: 2.1522
Epoch 2/5
443/443 [==============================] - 0s 210us/step - loss: 1.9198
Epoch 3/5
443/443 [==============================] - 0s 213us/step - loss: 1.6983
Epoch 4/5
443/443 [==============================] - 0s 208us/step - loss: 1.4609
Epoch 5/5
443/443 [==============================] - 0s 211us/step - loss: 1.2391
Epoch 1/5
664/664 [=======

1106/1106 [==============================] - 0s 249us/step - loss: 1.1779
Epoch 4/5
1106/1106 [==============================] - 0s 244us/step - loss: 1.1855
Epoch 5/5
1106/1106 [==============================] - 0s 246us/step - loss: 1.1958
Epoch 1/5
222/222 [==============================] - 2s 9ms/step - loss: 3.4393
Epoch 2/5
222/222 [==============================] - 0s 261us/step - loss: 3.1226
Epoch 3/5
222/222 [==============================] - 0s 273us/step - loss: 2.7199
Epoch 4/5
222/222 [==============================] - 0s 262us/step - loss: 1.8671
Epoch 5/5
222/222 [==============================] - 0s 247us/step - loss: 1.6116
Epoch 1/5
443/443 [==============================] - 2s 5ms/step - loss: 2.1917
Epoch 2/5
443/443 [==============================] - 0s 249us/step - loss: 1.8986
Epoch 3/5
443/443 [==============================] - 0s 249us/step - loss: 1.6935
Epoch 4/5
443/443 [==============================] - 0s 256us/step - loss: 1.4735
Epoch 5/5
443/443 [=====

Epoch 3/5
1106/1106 [==============================] - 0s 293us/step - loss: 1.1820
Epoch 4/5
1106/1106 [==============================] - 0s 290us/step - loss: 1.1422
Epoch 5/5
1106/1106 [==============================] - 0s 288us/step - loss: 1.1537
Epoch 1/5
222/222 [==============================] - 3s 12ms/step - loss: 1.2176
Epoch 2/5
222/222 [==============================] - 0s 305us/step - loss: 1.1368
Epoch 3/5
222/222 [==============================] - 0s 296us/step - loss: 1.0446
Epoch 4/5
222/222 [==============================] - 0s 305us/step - loss: 0.9715
Epoch 5/5
222/222 [==============================] - 0s 290us/step - loss: 0.9719
Epoch 1/5
443/443 [==============================] - 3s 6ms/step - loss: 0.9407
Epoch 2/5
443/443 [==============================] - 0s 282us/step - loss: 0.8429
Epoch 3/5
443/443 [==============================] - 0s 280us/step - loss: 0.8174
Epoch 4/5
443/443 [==============================] - 0s 287us/step - loss: 0.7982
Epoch 5/5
443

...

222/222 [==============================] - 0s 2ms/step - loss: 0.6814
Epoch 19/30
222/222 [==============================] - 0s 1ms/step - loss: 0.6502
Epoch 20/30
222/222 [==============================] - 0s 2ms/step - loss: 0.6665
Epoch 21/30
222/222 [==============================] - 0s 2ms/step - loss: 0.6212
Epoch 22/30
222/222 [==============================] - 0s 2ms/step - loss: 0.6420
Epoch 23/30
222/222 [==============================] - 0s 2ms/step - loss: 0.5799
Epoch 24/30
222/222 [==============================] - 0s 2ms/step - loss: 0.5752
Epoch 25/30
222/222 [==============================] - 0s 2ms/step - loss: 0.5915
Epoch 26/30
222/222 [==============================] - 0s 2ms/step - loss: 0.5966
Epoch 27/30
222/222 [==============================] - 0s 1ms/step - loss: 0.6294
Epoch 28/30
222/222 [==============================] - 0s 1ms/step - loss: 0.5909
Epoch 29/30
222/222 [==============================] - 0s 1ms/step - loss: 0.5377
Epoch 30/30
222/222 [=======

885/885 [==============================] - 1s 1ms/step - loss: 0.5284
Epoch 29/30
885/885 [==============================] - 1s 1ms/step - loss: 0.5098
Epoch 30/30
885/885 [==============================] - 1s 1ms/step - loss: 0.5084
Epoch 1/30
1106/1106 [==============================] - 63s 57ms/step - loss: 0.6579
Epoch 2/30
1106/1106 [==============================] - 2s 2ms/step - loss: 0.6355
Epoch 3/30
1106/1106 [==============================] - 2s 2ms/step - loss: 0.6289
Epoch 4/30
1106/1106 [==============================] - 2s 2ms/step - loss: 0.6206
Epoch 5/30
1106/1106 [==============================] - 2s 2ms/step - loss: 0.6201
Epoch 6/30
1106/1106 [==============================] - 2s 2ms/step - loss: 0.6131
Epoch 7/30
1106/1106 [==============================] - 2s 2ms/step - loss: 0.6214
Epoch 8/30
1106/1106 [==============================] - 2s 2ms/step - loss: 0.6253
Epoch 9/30
1106/1106 [==============================] - 2s 2ms/step - loss: 0.6162
Epoch 10/30
1106/

664/664 [==============================] - 1s 1ms/step - loss: 0.7117
Epoch 8/30
664/664 [==============================] - 1s 1ms/step - loss: 0.7196
Epoch 9/30
664/664 [==============================] - 1s 1ms/step - loss: 0.7072
Epoch 10/30
664/664 [==============================] - 1s 1ms/step - loss: 0.6984
Epoch 11/30
664/664 [==============================] - 1s 1ms/step - loss: 0.7011
Epoch 12/30
664/664 [==============================] - 1s 1ms/step - loss: 0.7246
Epoch 13/30
664/664 [==============================] - 1s 1ms/step - loss: 0.7032
Epoch 14/30
664/664 [==============================] - 1s 1ms/step - loss: 0.7032
Epoch 15/30
664/664 [==============================] - 1s 1ms/step - loss: 0.6936
Epoch 16/30
664/664 [==============================] - 1s 1ms/step - loss: 0.6756
Epoch 17/30
664/664 [==============================] - 1s 1ms/step - loss: 0.6926
Epoch 18/30
664/664 [==============================] - 1s 1ms/step - loss: 0.6689
Epoch 19/30
664/664 [=========

222/222 [==============================] - 0s 2ms/step - loss: 0.6778
Epoch 17/30
222/222 [==============================] - 0s 2ms/step - loss: 0.6940
Epoch 18/30
222/222 [==============================] - 0s 2ms/step - loss: 0.8554
Epoch 19/30
222/222 [==============================] - 0s 2ms/step - loss: 0.7541
Epoch 20/30
222/222 [==============================] - 0s 2ms/step - loss: 0.6617
Epoch 21/30
222/222 [==============================] - 0s 2ms/step - loss: 0.7385A: 0s - loss: 0.7
Epoch 22/30
222/222 [==============================] - 0s 2ms/step - loss: 0.6089
Epoch 23/30
222/222 [==============================] - 0s 2ms/step - loss: 0.6760
Epoch 24/30
222/222 [==============================] - 0s 2ms/step - loss: 0.6762
Epoch 25/30
222/222 [==============================] - 0s 2ms/step - loss: 0.6790
Epoch 26/30
222/222 [==============================] - 0s 2ms/step - loss: 0.6167
Epoch 27/30
222/222 [==============================] - 0s 2ms/step - loss: 0.6276
Epoch 28/30

885/885 [==============================] - 1s 2ms/step - loss: 0.5645
Epoch 27/30
885/885 [==============================] - 1s 2ms/step - loss: 0.5604
Epoch 28/30
885/885 [==============================] - 1s 1ms/step - loss: 0.5509
Epoch 29/30
885/885 [==============================] - 1s 1ms/step - loss: 0.5472
Epoch 30/30
885/885 [==============================] - 1s 1ms/step - loss: 0.5470


In [ ]:
columns=['Regressor','Score','BestEstimator']

In [ ]:
data_list=[]
data_row=['MLP',
                grid.best_score_,
                grid.best_estimator_
                ]
data_list.append(data_row)
score=pd.DataFrame(data_list,columns=columns)
score.set_index(['Regressor'],inplace=True)

In [ ]:
score

In [ ]:
grid.best_params_